In [ ]:
%python -m spacy download de_core_news_sm
%python -m spacy download en_core_web_sm
#%pip install -r requirements.txt

In [5]:
import os 
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax,pad
import math 
import copy
import time 
from torch.optim.lr_scheduler import LambdaLR

import pandas as pd 
import altair as alt


from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings 
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


warnings.filterwarnings("ignore")
RUN_EXAMPLES=True

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject


In [8]:
# 整个notebook的便利函数
def is_interactive_notebook():
    return __name__=="__main__"

def show_example(fn,args=[]):
    if __name__=="__main__" and RUN_EXAMPLES:
        return fn(*args)

def execute_example(fn,args=[]):
    if __name__=="__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups=[{"lr":0}]
        None
    def step(self):
        None
    def zero_grad(self,set_to_none=False):
        None

class DummyScheduler:
    def step(self):
        None        

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self,encoder,decoder,src_embed,tgt_embed,generator) -> None:
        super(EncoderDecoder,self).__init__()
        self.encoder=encoder
        self.decoder=decoder
        self.src_embed=src_embed
        self.tgt_embed=tgt_embed
        self.generator=generator

    def foward(self,src,tgt,src_mask,tgt_mask):
         return self.decoder(self.encode(src,src_mask),src_mask,tgt,tgt_mask)
    #      "Take in and process masked src and target sequences."
    def encode(self,src,src_mask):
        return self.encoder(self.src_embed(src),src_mask)
    
    def decoder(self,memory,src_mask.tgt,tgt_mask):
        